In [9]:
import numpy as np
from openai import AzureOpenAI
from pprint import pprint
from utils.prompts.prompt import *
from utils.prompts.Schema_Prompt_v1 import *

In [7]:
def calculate_confidence_score(logprobs: list) -> float:
    probs = []
    # iterate through the token predictions and get the linear probs [0;1]
    for logprob in logprobs:
        probs.append(np.round(np.exp(logprob)*100, 2))
    # calculate the confidence
    return round(np.mean(probs), 2)

In [2]:
client = AzureOpenAI(
    api_key = "823ee8f13b594481a9c23dbb62a60fb8",
    api_version = "2024-03-01-preview",
    azure_endpoint = "https://sql-chat-agent.openai.azure.com/"
)

In [3]:
QUERY = "Forgalomban van a novocetrin?"

In [4]:
sql_gen_messages = [
                {"role": "system", "content" : sql_system_message.format(system_prompt=SQL_SYSTEM_PROMPT, 
                                                                    table_info=SCHEMA_PROMPT)},
                {"role": "user", "content": question_message.format(question=QUERY)}
                ]

In [5]:
full_response = client.chat.completions.create(
    model="TextToSql_Pupha",
    messages=sql_gen_messages,
    temperature=0.0,
    max_tokens=1024,
    logprobs=True
)
pprint(full_response.choices[0].message.content)

("SELECT TOP 10 * FROM GYOGYSZ WHERE OEP_NEV LIKE '%novocetrin%' AND "
 'FORGALOMBAN = 1')


In [6]:
pprint(full_response)

ChatCompletion(id='chatcmpl-9EwUhJqFHDRXiCRAFiqZ9G4jNaRGx', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='SELECT', bytes=[83, 69, 76, 69, 67, 84], logprob=-0.0049804244, top_logprobs=[]), ChatCompletionTokenLogprob(token=' TOP', bytes=[32, 84, 79, 80], logprob=-0.55307436, top_logprobs=[]), ChatCompletionTokenLogprob(token=' ', bytes=[32], logprob=-0.00089180964, top_logprobs=[]), ChatCompletionTokenLogprob(token='10', bytes=[49, 48], logprob=-1.50940705e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token=' *', bytes=[32, 42], logprob=-0.16820113, top_logprobs=[]), ChatCompletionTokenLogprob(token=' FROM', bytes=[32, 70, 82, 79, 77], logprob=-0.009937405, top_logprobs=[]), ChatCompletionTokenLogprob(token=' G', bytes=[32, 71], logprob=-0.00095362135, top_logprobs=[]), ChatCompletionTokenLogprob(token='Y', bytes=[89], logprob=-3.1281633e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='OG', bytes=[79, 71], lo

In [10]:
logprobs = [x.logprob for x in full_response.choices[0].logprobs.content]
calculate_confidence_score(logprobs)

95.25